# Lesson 6: Planning and Stock Report Generation

## Setup

In [1]:
llm_config={"model": "gpt-4-turbo"}

## The task!

In [2]:
task = "Write a blogpost about the stock price performance of "\
"Nvidia in the past month. Today's date is 2024-04-23."

## Build a group chat

This group chat will include these agents:

1. **User_proxy** or **Admin**: to allow the user to comment on the report and ask the writer to refine it.
2. **Planner**: to determine relevant information needed to complete the task.
3. **Engineer**: to write code using the defined plan by the planner.
4. **Executor**: to execute the code written by the engineer.
5. **Writer**: to write the report.

In [3]:
import autogen

In [4]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

In [5]:
planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Planner. Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

In [6]:
engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="An engineer that writes code based on the plan "
    "provided by the planner.",
)

**Note**: In this lesson, you'll use an alternative method of code execution by providing a dict config. However, you can always use the LocalCommandLineCodeExecutor if you prefer. For more details about code_execution_config, check this: https://microsoft.github.io/autogen/docs/reference/agentchat/conversable_agent/#__init__

In [7]:
executor = autogen.ConversableAgent(
    name="Executor",
    system_message="Execute the code written by the "
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [8]:
writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer."
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="Writer."
    "Write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog."
)

## Define the group chat

In [9]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
)

In [10]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)


## Start the group chat!

<p style="background-color:#ECECEC; padding:15px; "> <b>Note:</b> In this lesson, you will use GPT 4 for better results. Please note that the lesson has a quota limit. If you want to explore the code in this lesson further, we recommend trying it locally with your own API key.

In [11]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Write a blogpost about the stock price performance of Nvidia in the past month. Today's date is 2024-04-23.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To begin writing a blog post about the stock price performance of Nvidia over the past month, we'll need to gather specific information. Here's a step-by-step outline of the required data and how to retrieve it using Python:

### Required Information
1. **Historical Stock Prices of Nvidia**: Obtain daily closing prices for Nvidia from today's date back to the last month.
2. **Significant Trading Volumes**: Check for any days within the past month where trading volume was notably higher or lower than average.
3. **Relevant News or Events**: Collect any significant news or corporate events that might have influenced stock price movements in the past month.

### Data Collection
#### Step 1: Historical Stock Prices
- Too


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Writer (to chat_manager):

```md
# Analyzing Nvidia's Recent Stock Performance: A Deep Dive Into Market Dynamics and Future Prospects

Nvidia's position at the forefront of the technology sector, particularly in areas such as artificial intelligence, gaming, and data centers, makes it a bellwether for investors tracking market trends. Over the last month, Nvidia's stock has demonstrated notable volatility, influenced by a combination of corporate announcements, industry-wide trends, and broader economic conditions. This blog post will dissect the stock performance of Nvidia from December 18, 2024, to January 16, 2025, to unearth the undercurrents driving these fluctuations and provide a well-rounded insight into Nvidia's future prospects.

## Overview of Stock Price Movements

Starting on December 18, 2024, Nvidia's stock price was at $128.91. The stock exhibited a general up

## Add a speaker selection policy

In [12]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message="Give the task, and send "
    "instructions to writer to refine the blog post.",
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)

planner = autogen.ConversableAgent(
    name="Planner",
    system_message="Given a task, please determine "
    "what information is needed to complete the task. "
    "Please note that the information will all be retrieved using"
    " Python code. Please only suggest information that can be "
    "retrieved using Python code. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps. If a step fails, try to "
    "workaround",
    description="Given a task, determine what "
    "information is needed to complete the task. "
    "After each step is done by others, check the progress and "
    "instruct the remaining steps",
    llm_config=llm_config,
)

engineer = autogen.AssistantAgent(
    name="Engineer",
    llm_config=llm_config,
    description="Write code based on the plan "
    "provided by the planner.",
)

writer = autogen.ConversableAgent(
    name="Writer",
    llm_config=llm_config,
    system_message="Writer. "
    "Please write blogs in markdown format (with relevant titles)"
    " and put the content in pseudo ```md``` code block. "
    "You take feedback from the admin and refine your blog.",
    description="After all the info is available, "
    "write blogs based on the code execution results and take "
    "feedback from the admin to refine the blog. ",
)

executor = autogen.ConversableAgent(
    name="Executor",
    description="Execute the code written by the "
    "engineer and report the result.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
)

In [13]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [user_proxy, executor],
        writer: [user_proxy, planner],
        executor: [user_proxy, engineer, planner],
        planner: [user_proxy, engineer, writer],
    },
    speaker_transitions_type="allowed",
)

In [14]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)

groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Write a blogpost about the stock price performance of Nvidia in the past month. Today's date is 2024-04-23.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To begin writing a blog post about the stock price performance of Nvidia over the past month, we'll need to gather specific information. Here's a step-by-step outline of the required data and how to retrieve it using Python:

### Required Information
1. **Historical Stock Prices of Nvidia**: Obtain daily closing prices for Nvidia from today's date back to the last month.
2. **Significant Trading Volumes**: Check for any days within the past month where trading volume was notably higher or lower than average.
3. **Relevant News or Events**: Collect any significant news or corporate events that might have influenced stock price movements in the past month.

### Data Collection
#### Step 1: Historical Stock Prices
- Too


--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Writer (to chat_manager):

Now let's move forward with fetching relevant news or events for Nvidia around the date identified with significant trading volume changes (January 7, 2025). This step involves using the NewsAPI, which will require an API key from NewsAPI.org. Please ensure you replace `YOUR_API_KEY_HERE` with your acquired API key.

Here's the Python script you need to execute:

```python
# filename: fetch_nvidia_news.py
from newsapi import NewsApiClient
import datetime

# Initialize the NewsAPI client
newsapi = NewsApiClient(api_key='YOUR_API_KEY_HERE')

# Define the date of interest
date_of_interest = datetime.date(2025, 1, 7)

# Fetch news articles about Nvidia for the date with significant trading volume
all_articles = newsapi.get_everything(q='Nvidia',
                                      from_param=date_of_interest,
                                      to=d

**Note**: You might experience slightly different interactions between the agents. The engineer agent may write incorrect code, which the executor agent will report and send back for correction. This process could go through multiple rounds.